**Chapter 5 — Transactions**

## What this chapter explores
- The structure of Bitcoin transactions (version, inputs, outputs, locktime).
- How inputs reference previous UTXOs and outputs create new UTXOs.
- How transaction data is serialized and parsed from raw hex.
- The role of `scriptSig` (unlocking script) and `scriptPubKey` (locking script).
- How varints are used to encode variable-length fields.
- How transaction fees are calculated (sum of inputs – sum of outputs).
- Writing code to parse and represent Tx, TxIn, and TxOut objects.
- Understanding how nodes and miners interpret and validate transactions.

## What we should learn by the end
- Be able to parse a raw Bitcoin transaction into its components.
- Understand the UTXO model and how inputs and outputs connect.
- Read and interpret ScriptSig and ScriptPubKey fields at a high level.
- Implement transaction fee calculation in code.
- Gain insight into how miners select transactions based on fees.
- Build the foundation for later chapters that cover signing and validating transactions.

In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc
import helper
import script
import tx

### Exercise 1

Write the version parsing part of the `parse` method that we've defined. To do this properly, you'll have to convert 4 bytes into a Little-Endian integer.

#### Make [this test](/edit/code-ch05/tx.py) pass: `tx.py:TxTest:test_parse_version`

__________________________

Difficult to find where this is in the `tx.py` notebook...

Check the parse class. 

We need to use little endian method from previous chapter. 

`TxTest("test_parse_version")` uses
`raw_tx = bytes.fromhex('0100000001813f....')`

`stream = BytesIO(raw_tx)` creates a file-like object so the data can be read sequentially, instead of reading it in hex format. 

In `parse`, call `stream.read(4)` to get the first 4 bytes, then convert with `little_endian_to_int`. This gives the version number, which is returned as part of the `Tx` class.

Little-endian is required because Bitcoin transaction fields are stored in that format.



In [2]:
# Exercise 1

reload(tx)
run(tx.TxTest("test_parse_version"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


How to get a `Script` object from hexadecimal in Python:

In [3]:
from io import BytesIO
from script import Script
script_hex = ('6b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')
stream = BytesIO(bytes.fromhex(script_hex))
script_sig = Script.parse(stream)
print(script_sig)

3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed01 0349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a


### Exercise 2

Write the inputs parsing part of the `parse` method in `Tx` and the `parse` method for `TxIn`.

#### Make [this test](/edit/code-ch05/tx.py) pass: `tx.py:TxTest:test_parse_inputs`

_______________________


1. **Read previous transaction**

   * `s.read(32)[::-1]` → takes 32 bytes, reverses them because Bitcoin stores txids in little-endian.

2. **Read previous index**

   * `s.read(4)` → 4 bytes, converted to integer little-endian with `little_endian_to_int`.
   * This tells us which output of the previous transaction is being spent.

3. **Parse ScriptSig**

   * `Script.parse(s)` → reads a varint (script length), then that many bytes, and parses them into a `Script` object.

4. **Read sequence**

   * `s.read(4)` → 4 bytes, converted to integer little-endian.
   * Usually `0xffffffff` unless replaced by locktime logic.

5. **Return TxIn object**

   * Wraps everything (`prev_tx`, `prev_index`, `script_sig`, `sequence`) in a `TxIn` instance.

---

* 32 bytes = txid
* 4 bytes = output index
* varint + script bytes = ScriptSig
* 4 bytes = sequence
* return `TxIn`.


In [4]:
# Exercise 2

reload(tx)
run(tx.TxTest("test_parse_inputs"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 3

Write the outputs parsing part of the `parse` method in `Tx` and the `parse` method for `TxOut`.

#### Make [this test](/edit/code-ch05/tx.py) pass: `tx.py:TxTest:test_parse_outputs`

__________________

Testing the output parsing: 

* read_varint(s) is called twice: once for inputs and once for outputs. This is necessary because the number of inputs and outputs can be different in each transaction. The first call tells us how many TxIn objects (spending previous UTXOs) to parse, and the second call tells us how many TxOut objects (new UTXOs/payments) to parse.

**Example**

Suppose Alice received 0.5 BTC in an earlier transaction (that output is now a UTXO belonging to Alice).
If Alice wants to pay Bob:

Her new transaction will include 1 input (TxIn) pointing back to that 0.5 BTC UTXO.

That input is spending the previous UTXO.

* parsing the number of outputs provides the number of outbound satoshis and the pubkey.

* Parsing the number of outputs lets us know how many TxOut objects to create, i.e. how many “payments” this transaction makes.

In [5]:
# Exercise 3

reload(tx)
run(tx.TxTest("test_parse_outputs"))

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Exercise 4

Write the Locktime parsing part of the `parse` method in `Tx`.

#### Make [this test](/edit/code-ch05/tx.py) pass: `tx.py:TxTest:test_parse_locktime`

______

the parser is using the logic we've defined previously for the version, inputs and outputs. So it knows from that the locktime is the last 4 bytes. 

**Locktime parsing notes**

* The parser reads fields in order: version → inputs → outputs.
* After those are consumed, only 4 bytes remain at the end of the transaction.
* By definition, those last 4 bytes are the locktime.
* s.read(4) + little_endian_to_int gives the locktime value.

In [6]:
# Exercise 4

reload(tx)
run(tx.TxTest("test_parse_locktime"))

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### Exercise 5

What is the ScriptSig from the second input, ScriptPubKey from the first output and the amount of the second output for this transaction?

```
010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e0100
00006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951
c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0
da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4
038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a473044022078
99531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b84
61cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba
1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c35
6efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da
6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c3
4210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49
abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd
04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea833
1ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c
2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20df
e7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948
a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46
430600
```

In [7]:
# Exercise 5

from io import BytesIO
from tx import Tx

hex_transaction = '010000000456919960ac691763688d3d3bcea9ad6ecaf875df5339e148a1fc61c6ed7a069e010000006a47304402204585bcdef85e6b1c6af5c2669d4830ff86e42dd205c0e089bc2a821657e951c002201024a10366077f87d6bce1f7100ad8cfa8a064b39d4e8fe4ea13a7b71aa8180f012102f0da57e85eec2934a82a585ea337ce2f4998b50ae699dd79f5880e253dafafb7feffffffeb8f51f4038dc17e6313cf831d4f02281c2a468bde0fafd37f1bf882729e7fd3000000006a47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937feffffff567bf40595119d1bb8a3037c356efd56170b64cbcc160fb028fa10704b45d775000000006a47304402204c7c7818424c7f7911da6cddc59655a70af1cb5eaf17c69dadbfc74ffa0b662f02207599e08bc8023693ad4e9527dc42c34210f7a7d1d1ddfc8492b654a11e7620a0012102158b46fbdff65d0172b7989aec8850aa0dae49abfb84c81ae6e5b251a58ace5cfeffffffd63a5e6c16e620f86f375925b21cabaf736c779f88fd04dcad51d26690f7f345010000006a47304402200633ea0d3314bea0d95b3cd8dadb2ef79ea8331ffe1e61f762c0f6daea0fabde022029f23b3e9c30f080446150b23852028751635dcee2be669c2a1686a4b5edf304012103ffd6f4a67e94aba353a00882e563ff2722eb4cff0ad6006e86ee20dfe7520d55feffffff0251430f00000000001976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac005a6202000000001976a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac46430600'

# convert the hex_transaction to binary

# create a stream using BytesIO
# use Tx.parse to get the transaction object.
# ScriptSig from second input
# ScriptPubKey from first output
# Amount from second output

In [8]:
stream = BytesIO(bytes.fromhex(hex_transaction))
transaction = Tx.parse(stream)

In [9]:
transaction

tx: ee51510d7bbabe28052038d1deb10c03ec74f06a79e21913c6fcf48d56217c87
version: 1
tx_ins:
9e067aedc661fca148e13953df75f8ca6eada9ce3b3d8d68631769ac60999156:1
d37f9e7282f81b7fd3af0fde8b462a1c28024f1d83cf13637ec18d03f4518feb:0
75d7454b7010fa28b00f16cccb640b1756fd6e357c03a3b81b9d119505f47b56:0
45f3f79066d251addc04fd889f776c73afab1cb22559376ff820e6166c5e3ad6:1
tx_outs:
1000273:OP_DUP OP_HASH160 ab0c0b2e98b1ab6dbf67d4750b0a56244948a879 OP_EQUALVERIFY OP_CHECKSIG
40000000:OP_DUP OP_HASH160 3c82d7df364eb6c75be8c80df2b3eda8db573970 OP_EQUALVERIFY OP_CHECKSIG
locktime: 410438

In [10]:
print(transaction.tx_ins[1].script_sig.raw_serialize().hex())

47304402207899531a52d59a6de200179928ca900254a36b8dff8bb75f5f5d71b1cdc26125022008b422690b8461cb52c3cc30330b23d574351872b7c361e9aae3649071c1a7160121035d5c93d9ac96881f19ba1f686f15f009ded7c62efe85a872e6a19b43c15a2937


In [11]:
print(transaction.tx_outs[0].script_pubkey.serialize().hex())

1976a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac


In [12]:
print(transaction.tx_outs[1].amount)

40000000


In [13]:
for i, tx_out in enumerate(transaction.tx_outs):
    print(f"Output {i}: {tx_out.amount} sats ({tx_out.amount / 100_000_000:.8f} BTC)")
    print(f"  ScriptPubKey: {tx_out.script_pubkey.raw_serialize().hex()}\n")


Output 0: 1000273 sats (0.01000273 BTC)
  ScriptPubKey: 76a914ab0c0b2e98b1ab6dbf67d4750b0a56244948a87988ac

Output 1: 40000000 sats (0.40000000 BTC)
  ScriptPubKey: 76a9143c82d7df364eb6c75be8c80df2b3eda8db57397088ac



### Exercise 6

Write the `fee` method for the `Tx` class.

#### Make [this test](/edit/code-ch05/tx.py) pass: `tx.py:TxTest:test_fee`

_____
transaction fees incentivise miners to include the transactions in the blockspace or mempool. Only transactions in the blockspace are part of the blockchain and final. 

What happens if miners don't include transactions? Will they start taking control of the blockchain? 

The transaction fee is the sum of the inputs minus the outputs

In [18]:
# Exercise 6

reload(tx)
run(tx.TxTest("test_fee"))

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


# Chapter review


## Structure of a Transaction

- **Version** (4 bytes, little-endian): protocol version.
- **Inputs (TxIn):**
  - `prev_tx` (32 bytes, txid of UTXO being spent, little-endian).
  - `prev_index` (4 bytes, which output of that tx).
  - `scriptSig` (unlocking script, proves spending rights).
  - `sequence` (4 bytes, often `0xffffffff`).
- **Outputs (TxOut):**
  - `amount` (8 bytes, satoshis).
  - `scriptPubKey` (locking script, defines spending condition).
- **Locktime** (4 bytes, little-endian): earliest block/time the tx is valid.

## Key Concepts

- **UTXO Model**
  - Inputs reference previous unspent outputs.
  - Outputs create new UTXOs.
- **Varints**
  - Used for number of inputs, number of outputs, and script lengths.
- **Scripts**
  - `scriptPubKey` = locking conditions.
  - `scriptSig` = unlocking data.
  - Together must evaluate to *true*.
- **Transaction Fee**
  - Fee = sum(inputs) – sum(outputs).

## Code/Parsing Flow

- `Tx.parse()`: version → inputs → outputs → locktime.
- `TxIn.parse()`: prev_tx, prev_index, scriptSig, sequence.
- `TxOut.parse()`: amount, scriptPubKey.
- `Tx.fee()`: sum of input values – sum of outputs.

## Big Picture

- Transactions are inputs → outputs (UTXOs consumed and created).
- Nodes check that scripts unlock UTXOs correctly.
- Miners prioritize by fee rate (sats per byte).
- You now have a working transaction parser and fee calculator.

## Takeaway
Bitcoin transactions = UTXOs being spent and created, locked/unlocked by scripts, with fees equal to leftover satoshis.
